In [12]:
#Mount your drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Move to HPSG model directory
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser-final/HPSG-Neural-Parser-master

/content/drive/.shortcut-targets-by-id/1DP82lkt284bhnm7X9kA_LML9IgUxQIL9/Khoa_luan_tot_nghiep/HPSG-Parser-final/HPSG-Neural-Parser-master


Before training, a executable file *evalb* is needed. This file will be called from Python for evaluation.

In [14]:
#Move to folder EVALB to begin compile "evalb" file
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser-final/HPSG-Neural-Parser-master/EVALB

/content/drive/.shortcut-targets-by-id/1DP82lkt284bhnm7X9kA_LML9IgUxQIL9/Khoa_luan_tot_nghiep/HPSG-Parser-final/HPSG-Neural-Parser-master/EVALB


Run this command to compile *evalb* executable file.

In [15]:
!make

make: Nothing to be done for 'all'.


In [17]:
# Move back to project directory
%cd /content/drive/My Drive/Khoa_luan_tot_nghiep/HPSG-Parser-final/HPSG-Neural-Parser-master

/content/drive/.shortcut-targets-by-id/1DP82lkt284bhnm7X9kA_LML9IgUxQIL9/Khoa_luan_tot_nghiep/HPSG-Parser-final/HPSG-Neural-Parser-master


Run this command to allow Google Colab to access the *evalb* executable file

In [ ]:
!chmod 755 -R EVALB/evalb

Run these command to install required packages.

In [ ]:
!pip3 install torch torchvision torchaudio
!pip install allennlp==0.9.0
!pip install pytorch-transformers
!pip install cython

Run this command to begin training HPSG model. You should configure settings in file run_single.sh in project folder before training.

In [20]:
!sh run_single.sh

Streaming output truncated to the last 5000 lines.
epoch 6 batch 5/36 processed 46,135 batch-loss 82.4278 grad-norm 137.8719 epoch-elapsed 0h00m26s total-elapsed 0h17m26s
epoch 6 batch 6/36 processed 46,385 batch-loss 83.5256 grad-norm 115.9085 epoch-elapsed 0h00m32s total-elapsed 0h17m32s
epoch 6 batch 7/36 processed 46,635 batch-loss 82.5822 grad-norm 142.3882 epoch-elapsed 0h00m37s total-elapsed 0h17m37s
epoch 6 batch 8/36 processed 46,885 batch-loss 86.8155 grad-norm 146.2888 epoch-elapsed 0h00m42s total-elapsed 0h17m42s
epoch 6 batch 9/36 processed 47,135 batch-loss 79.2548 grad-norm 70.5267 epoch-elapsed 0h00m48s total-elapsed 0h17m48s
dev-fscore (Recall=6.63, Precision=98.00, FScore=12.42) dev-elapsed 0h00m01s total-elapsed 0h17m49s
W. Punct: ucorr: 951, lcorr: 13, total: 4260, uas: 22.32%, las: 0.31%, ucm: 2.50%, lcm: 0.00%
Wo Punct: ucorr: 951, lcorr: 13, total: 4260, uas: 22.32%, las: 0.31%, ucm: 2.50%, lcm: 0.00%
Root: corr: 133, total: 200, acc: 66.50%
epoch 6 batch 10/36 p

Run this command to evaluate your trained HPSG model. To change input parameter settings, check file test.sh in project folder.

In [21]:
!sh test.sh

Loading model from models/run1st/joint_single_best_dev=78.57_devuas=85.42_devlas=79.23.pt...
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
Reading dependency parsing data from data/VnData/VnDTv1.1-predicted-test.conll
Loading test trees from data/VnData/test.clean...
Loaded 1,020 test examples.
Parsing test sentences...
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
test-fscore (Recall=79.77, Precision=78.66, FScore=

Run this command to parse the sentences. Change settings in file parse.sh in project folder.

In [ ]:
!sh parse.sh